In [1]:
import pickle
import torch
from transformers import SpeechT5ForSpeechToText, SpeechT5ForTextToSpeech

In [2]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device
device =torch.device("cpu")

In [3]:
#model_asr = SpeechT5ForSpeechToText.from_pretrained("microsoft/speecht5_asr").to(device)
#model_orig = SpeechT5ForSpeechToText.from_pretrained("microsoft/speecht5_asr").to(device)
model_tts = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)
ckpt = torch.load("../checkpoints/speecht5_base.pt", map_location=device)

# TEXT PRE-NET

In [9]:
for name, p in model_tts.named_parameters():
    if name.startswith("speecht5.encoder.prenet"):
        print(name, p.shape ,p)

speecht5.encoder.prenet.embed_tokens.weight torch.Size([81, 768]) Parameter containing:
tensor([[ 0.2236,  0.1462,  0.2703,  ..., -0.5044, -0.3403, -0.2323],
        [-0.0267, -0.0211,  0.0806,  ...,  0.0826, -0.0250,  0.0142],
        [ 0.1836,  0.1970,  0.1066,  ..., -0.0246,  0.0242, -0.1271],
        ...,
        [-0.0618, -0.0311,  0.0687,  ...,  0.1088, -0.1075, -0.0759],
        [-0.0393, -0.1295,  0.0012,  ...,  0.0482, -0.0222, -0.3157],
        [-0.0368, -0.0320,  0.0710,  ...,  0.1121, -0.0775, -0.0605]],
       requires_grad=True)
speecht5.encoder.prenet.encode_positions.alpha torch.Size([]) Parameter containing:
tensor(0.1053, requires_grad=True)


In [10]:
for name, p in ckpt['model'].items():
    if name.startswith("text_encoder_prenet"):
        print(name, p.shape, p)

text_encoder_prenet.encoder_prenet.0.weight torch.Size([81, 768]) tensor([[ 0.2236,  0.1462,  0.2703,  ..., -0.5044, -0.3403, -0.2323],
        [-0.0267, -0.0211,  0.0806,  ...,  0.0826, -0.0250,  0.0142],
        [ 0.1836,  0.1970,  0.1066,  ..., -0.0246,  0.0242, -0.1271],
        ...,
        [-0.0473, -0.0490,  0.0584,  ...,  0.1119, -0.1022, -0.0686],
        [-0.0393, -0.1295,  0.0012,  ...,  0.0482, -0.0222, -0.3157],
        [-0.0368, -0.0320,  0.0710,  ...,  0.1121, -0.0775, -0.0605]])
text_encoder_prenet.encoder_prenet.1.alpha torch.Size([]) tensor(0.1270)


In [12]:
model_tts.speecht5.encoder.prenet.encode_positions.alpha, model_tts.speecht5.encoder.prenet.encode_positions.alpha.dtype

(Parameter containing:
 tensor(0.1053, requires_grad=True),
 torch.float32)

In [15]:
model_tts.speecht5.encoder.prenet.encode_positions.alpha = torch.nn.Parameter(0.1270, dtype=torch.float32)

AttributeError: 'float' object has no attribute 'detach'

# SPEECH PRE-NET

In [4]:
for name, phf in model_asr.named_parameters():
    if name.startswith("speecht5.encoder.prenet.feature_projection.layer_norm.weight"):
        print(name, phf[:5], phf.shape)
        break

speecht5.encoder.prenet.feature_projection.layer_norm.weight tensor([0.9146, 0.6802, 0.7935, 0.4436, 1.0410], device='cuda:0',
       grad_fn=<SliceBackward0>) torch.Size([512])


In [5]:
for name, p in ckpt['model'].items():
    if name.startswith("speech_encoder_prenet.layer_norm.weight"):
        print(name,p[:5], p.shape)
        print(torch.allclose(p, phf))

speech_encoder_prenet.layer_norm.weight tensor([0.9727, 0.7222, 0.8418, 0.4727, 1.1113], device='cuda:0') torch.Size([512])
False


In [5]:
prenet_mapping = {}

hf_roi = 2
bs_roi = 0
for name, p in model_asr.named_parameters():
    if name.startswith("speecht5.encoder.prenet"):
        #print(name, p.shape)
        splitt = name.split('.')
        for name_base, p_base in ckpt['model'].items():
            if name_base.startswith("speech_encoder_prenet"):
                splitt_base = name_base.split('.')
                
                if splitt[hf_roi+1] == "masked_spec_embed" and splitt_base[bs_roi+1] == "mask_emb":
                    prenet_mapping[name] = name_base
                    break
                elif "feature_projection.layer_norm.weight" in name and "layer_norm.weight" in name_base:
                    prenet_mapping[name] = name_base
                    break
                elif "feature_projection.layer_norm.bias" in name and "layer_norm.bias" in name_base:
                    prenet_mapping[name] = name_base
                    break
                elif "feature_projection.projection.weight" in name and "post_extract_proj.weight" in name_base:
                    prenet_mapping[name] = name_base
                    break
                elif "feature_projection.projection.bias" in name and "post_extract_proj.bias" in name_base:
                    prenet_mapping[name] = name_base
                    break
                elif "feature_encoder" in name and "feature_extractor" in name_base:
                    #print("Original", name_base)
                    encoder_name = "feature_encoder"
                    conv = "conv"
                    layer_norm = "layer_norm"
                    new_name_base = name_base.replace("feature_extractor", encoder_name).replace("0.weight","conv.weight").replace("2.weight", "layer_norm.weight").replace("2.bias","layer_norm.bias")
                    #print("New:", new_name_base)
                    #print(f"Splits {name.split('.',maxsplit=hf_roi+2)[-1]} =?= {new_name_base.split('.',maxsplit=hf_roi)[-1]}")
                    if name.split(".",maxsplit=hf_roi+2)[-1] == new_name_base.split(".",maxsplit=hf_roi)[-1]:
                        prenet_mapping[name] = name_base
                        break
                elif "pos_conv" in name and "pos_conv" in name_base:
                    new_name_base = name_base.replace("pos_conv", "pos_conv_embed").replace("0", "conv")
                    #print(f"Splits {name.split('.',maxsplit=hf_roi+1)[-1]} =?= {new_name_base.split('.',maxsplit=hf_roi-1)[-1]}")
                    if name.split(".",maxsplit=hf_roi+1)[-1] == new_name_base.split(".",maxsplit=hf_roi-1)[-1]:
                        #print("HERE")
                        prenet_mapping[name] = name_base
                        break 
                    
                    
prenet_mapping

{'speecht5.encoder.prenet.masked_spec_embed': 'speech_encoder_prenet.mask_emb',
 'speecht5.encoder.prenet.feature_encoder.conv_layers.0.conv.weight': 'speech_encoder_prenet.feature_extractor.conv_layers.0.0.weight',
 'speecht5.encoder.prenet.feature_encoder.conv_layers.0.layer_norm.weight': 'speech_encoder_prenet.feature_extractor.conv_layers.0.2.weight',
 'speecht5.encoder.prenet.feature_encoder.conv_layers.0.layer_norm.bias': 'speech_encoder_prenet.feature_extractor.conv_layers.0.2.bias',
 'speecht5.encoder.prenet.feature_encoder.conv_layers.1.conv.weight': 'speech_encoder_prenet.feature_extractor.conv_layers.1.0.weight',
 'speecht5.encoder.prenet.feature_encoder.conv_layers.2.conv.weight': 'speech_encoder_prenet.feature_extractor.conv_layers.2.0.weight',
 'speecht5.encoder.prenet.feature_encoder.conv_layers.3.conv.weight': 'speech_encoder_prenet.feature_extractor.conv_layers.3.0.weight',
 'speecht5.encoder.prenet.feature_encoder.conv_layers.4.conv.weight': 'speech_encoder_prenet.fea

In [6]:
prenet_dict = {}
for hf_layer, bs_layer in prenet_mapping.items():
    for name, p in ckpt['model'].items():
        if bs_layer == name:
            prenet_dict[hf_layer] = p
            break

In [7]:
prenet_dict_state = {}
for layer, param in prenet_dict.items():
    new_layer = layer.split('.',maxsplit=3)[-1]
    #print(new_layer)
    prenet_dict_state[new_layer] = param
    
for name, param in model_asr.named_parameters():
    if "pos_sinusoidal_embed.weights" in name:
        prenet_dict_state[name.split('.',maxsplit=3)[-1]] = param

In [8]:
model_asr.speecht5.encoder.prenet.load_state_dict(prenet_dict_state)

<All keys matched successfully>

In [12]:
for modified,original in zip(model_asr.speecht5.encoder.prenet.named_parameters(), model_orig.speecht5.encoder.prenet.named_parameters()):
    print(torch.allclose(modified[1],original[1],rtol=1e-04, atol=1e-06))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True


In [11]:
prenet_dict_state['feature_projection.layer_norm.weight'][:5]

tensor([0.9727, 0.7222, 0.8418, 0.4727, 1.1113], device='cuda:0')

In [13]:
model_asr.speecht5.encoder.prenet.feature_projection.layer_norm.weight[:5]

tensor([0.9727, 0.7222, 0.8418, 0.4727, 1.1113], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [14]:
model_orig.speecht5.encoder.prenet.feature_projection.layer_norm.weight[:5]

tensor([0.9146, 0.6802, 0.7935, 0.4436, 1.0410], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [11]:
"aaabaaa".replace("b","c").replace("a","b")

'bbbcbbb'

In [ ]:
'''
layer_type = splitt[hf_roi+1]
                    layer_layer = splitt[hf_roi+2]
                    layer_num = splitt[hf_roi+3]
                    layer_block = splitt[hf_roi+4]
                    layer_value = splitt[hf_roi+5]
                    
                    layer_type_base = splitt[bs_roi+1]
                    layer_layer_base = splitt[bs_roi+2]
                    layer_num_base = splitt[bs_roi+3]
                    layer_block_base = splitt[bs_roi+4]
                    layer_value_base = splitt[bs_roi+5]
'''

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
mapping_file = "../extracted/speecht5/mapping/encoder_state_dict.pickle"
with open(mapping_file, 'rb') as handle:
    encoder_state_dict = pickle.load(handle)

In [4]:
model = SpeechT5ForSpeechToText.from_pretrained("microsoft/speecht5_asr").to(device)

In [5]:
modelb = SpeechT5ForSpeechToText.from_pretrained("microsoft/speecht5_asr").to(device)

In [6]:
modelb.speecht5.encoder.wrapped_encoder.load_state_dict(encoder_state_dict)

<All keys matched successfully>

In [8]:
torch.allclose(torch.ones((2,2)), torch.ones(2,2))

True

In [21]:
for a,b in zip(model.speecht5.encoder.prenet.named_parameters(), modelb.speecht5.encoder.wrapped_encoder.named_parameters()):
    print(torch.allclose(a[1],b[1]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [12]:
model.speecht5.encoder.wrapped_encoder

SpeechT5Encoder(
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0-11): 12 x SpeechT5EncoderLayer(
      (attention): SpeechT5Attention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (feed_forward): SpeechT5FeedForward(
        (intermediate_dropout): Dropout(p=0.1, inplace=False)
        (intermediate_dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
        (output_dense): Linear(in_features=3072, out_features=768, bias=True)
        (output_dropout): Dropout(p=0.1